In [5]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [6]:
ingresos_totales_finales = []
costos_totales_finales = []
utilidad_total_final = []
demandas_por_mercado = []
suma_litros_peridos_toda_simulacion = 0
calidad_minima_por_simulacion = []
litros_totales_perdidos_etapa1 = 0
litros_totales_perdidos_etapa2 = []
litros_totales_perdidos_etapa2_tipo3 = []
cepa_inicio_total = []
C1_fermentacion_suma = 0
C2_fermentacion_suma = 0
C3_fermentacion_suma = 0
C4_fermentacion_suma = 0
C5_fermentacion_suma = 0
C6_fermentacion_suma = 0



### ETAPA 1: COSECHA ###

In [7]:
g = 0
sobra_a = 0
sobra_b = 0
sobra_c = 0
sobra_d = 0
veces_mercado_a_no_cumple = 0
veces_mercado_b_no_cumple = 0
veces_mercado_c_no_cumple = 0
veces_mercado_d_no_cumple = 0
suma_resto_mercado_a_positiva_solamente = 0
suma_resto_mercado_b_positiva_solamente = 0
suma_resto_mercado_c_positiva_solamente = 0
suma_resto_mercado_d_positiva_solamente = 0
for _ in range(1000):
    # parametros cosecha
    # parametros cosecha
    periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
    umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
    lotes = []
    penalizaciones = {
        'C1': [0.03, 0.02, 0.01, 0.005],
        'C2': [0.025, 0.012, 0.004, 0.002],
        'C3': [0.03, 0.02, 0.01, 0.005],
        'C4': [0.025, 0.012, 0.01, 0.002],
        'C5': [0.03, 0.02, 0.01, 0.005],
        'C6': [0.025, 0.012, 0.004, 0.002]
    }
    # Leo la base de datos y la guardo en una lista
    c.execute("SELECT * FROM lotes")
    lotes_obtenidos = c.fetchall()
    lotes.append(lotes_obtenidos)
    lotes = lotes[0]
    #Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

    # pasar lotes a lista de listas
    lotes = [list(elem) for elem in lotes]
    # Funcion que estima el dia de cosecha con media 0 y desv est 2
    def dia_estimado():
            optimal_day = np.random.normal(0, 2)
            return int(round(optimal_day))
    for lote in lotes:
        variacion_optimo = dia_estimado()
        lote[5]+=variacion_optimo
        lote.append(variacion_optimo)

    def calcular_probabilidad_lluvia(P_seco_a_lluvioso, P_lluvioso_a_lluvioso):
        # Inicializar probabilidades iniciales según el estado inicial deseado
        P_dia_seco = 1
        P_dia_lluvioso = 0
        dias = 7

        # Calcular probabilidades de estado para cada día
        for _ in range(dias):
            P_dia_seco_nuevo = (P_dia_seco * (1 - P_seco_a_lluvioso) + P_dia_lluvioso * (1 - P_lluvioso_a_lluvioso))
            P_dia_lluvioso_nuevo = (P_dia_seco * P_seco_a_lluvioso + P_dia_lluvioso * ( P_lluvioso_a_lluvioso))
            P_dia_seco, P_dia_lluvioso = P_dia_seco_nuevo, P_dia_lluvioso_nuevo

        porcentaje_lluvia = P_dia_lluvioso 
        
        return porcentaje_lluvia
    for lote in lotes:
        prob_lluvia = calcular_probabilidad_lluvia(lote[6],lote[7])
        prob_lluvia = round(prob_lluvia, 6)
        lote.append(prob_lluvia)
    # Lotes actualizados
    # Función para predecir el clima con markov
    def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
        clima_prediccion = []  # Lista para almacenar la predicción
        tipo_3 = []
        # Inicializar el primer día como seco
        clima_prediccion.append(0)
        tipo_3.append(0)
        # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
        for _ in range(7):
            if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
                clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
                tipo_3.append(0)
            else:  # Si el día anterior fue lluvioso
                clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
                if clima_prediccion[-1] == 1:
                    tipo_3.append(1 if random.random() <= 0.05 else 0)
        if sum(tipo_3) >= 1:
            perdida = True
        else:
            perdida = False
        return clima_prediccion, perdida
    def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
        lista_penalizaciones = penalizaciones[tipo_uva]
        index = 7
        while index != 0:
            if index == 7:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[0])
            elif index == 6:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[1])
            elif index == 5:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[2])
            elif index == 4:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 3:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 2:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 1:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            index -= 1
        return calidad

    def obtener_penalizaciones_2(tipo_uva, calidad, clima_prediccion):
        lista_penalizaciones = penalizaciones[tipo_uva]
        index = 5
        while index != 0:
            if index == 5:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[0])
            elif index == 4:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[1])
            elif index == 3:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[2])
            elif index == 2:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            elif index == 1:
                if clima_prediccion[index] == 1:
                    calidad = calidad * (1 - lista_penalizaciones[3])
            index -= 1
        return calidad


    # funcion que entrega el valor final de la calidad considerando la lluvia
    def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
        t_minus = -7
        t_plus = 7
        A = np.array([
            [t_minus**2, t_minus, 1],
            [0, 0, 1],
            [t_plus**2, t_plus, 1]])
        b = np.array([q_t_minus, 1, q_t_plus])
        coefficients = np.linalg.solve(A, b)
        a, b, c = coefficients
        def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
            # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
            calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
            calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
            return calidad_final
        q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
        return q_t_optimal

    def calcular_calidad_2(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
        t_minus = -7
        t_plus = 7
        A = np.array([
            [t_minus**2, t_minus, 1],
            [0, 0, 1],
            [t_plus**2, t_plus, 1]])
        b = np.array([q_t_minus, 1, q_t_plus])
        coefficients = np.linalg.solve(A, b)
        a, b, c = coefficients
        def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
            # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
            calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
            calidad_final = obtener_penalizaciones_2(tipo_uva, calidad_sin_lluvia, clima_prediccion)
            return calidad_final
        q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
        return q_t_optimal
    # Parametros para encontrar en la simulacion etapa 1
    calidades_lotes = []
    lotes_perdidos = []
    lotes_obtenidos = []
    litros_totales_perdidos = 0
    litros_totales_obtenidos = 0
    botellas_totales_perdidas = 0
    botellas_totales_obtenidas = 0
    costos_totales_perdidos = 0
    costos_totales_obtenidos = 0
    #Se crea ua funcion que nos entregue una lista que corresponde a la cantidad de lluvia maxima tolerada por cada una de las cepas
    #Esta funcion se puede alterar dependiendo de que % de lotes se quiere comprar con compra spot
    def tolerancia_cepa(lotes):
        cepas = ["C1","C2","C3","C4","C5","C6"]
        tolerancia = []

        penalizaciones = {'C1': [0.03, 0.02, 0.01, 0.005],
                            'C2': [0.025, 0.012, 0.004, 0.002],
                            'C3': [0.03, 0.02, 0.01, 0.005],
                            'C4': [0.025, 0.012, 0.01, 0.002],
                            'C5': [0.03, 0.02, 0.01, 0.005],
                            'C6': [0.025, 0.012, 0.004, 0.002]
                            }
        #Se hacen dos litas por cada cepa, donde una corresponde a la calidad o ponterador de calidad obtenido en base a una de las partes de la ecuacion de calidad
        #(la parte que pensaliza por la presencia de lluvias en cada uno de los dias previos)
        #Luego de eso se determina un porcentaje de los lotes que será comprado con contrato spot (Ej: el 20 porciento que contenga a los lotes con peor calidad)
        for cepa in cepas:
            valores_lluvia = []
            valores_calidad = []
            for lote in lotes:
                porcentaje_lluvia = lote[10]
                lote_cepa = lote[3]
                if cepa == lote_cepa:
                    calidad = ((1-penalizaciones[cepa][0]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][1]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][2]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                        (1-penalizaciones[cepa][3]*porcentaje_lluvia)
                        )
                    valores_lluvia.append(porcentaje_lluvia)
                    valores_calidad.append(calidad)
            
            #Se ordenan las listas de mayor a menor para luego poder obtener que valor corresponde al % deseado
            valores_lluvia.sort(reverse = True)
            valores_calidad.sort(reverse = True)

            #En este caso el 0.2 es porque buscamos la calidad en el 20% mas bajo y con su posicion encontraremos la lluvia que genera la calidad del 20% mas bajo
            posicion_porcentaje = int(len(valores_calidad) * 0.2)
            valor_lluvia = valores_lluvia[posicion_porcentaje]
            tolerancia.append(valor_lluvia)
        return tolerancia             


    #Creamos una funcion a la que se le entregue un lote y una lista con el porcentaje de lluvia tolerado por cepa y en base a eso
    #se decide que tipo de contrato se escogerá
    def tipo_contrato(tolerancia, lote):
        cepa = lote[3]
        porcentaje_lluvia = lote[10]
        if cepa == "C1":
            tolerancia_cepa = tolerancia[0]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C2":
            tolerancia_cepa = tolerancia[1]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C3":
            tolerancia_cepa = tolerancia[2]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0
            
        elif cepa == "C4":
            tolerancia_cepa = tolerancia[3]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C5":
            tolerancia_cepa = tolerancia[4]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        elif cepa == "C6":
            tolerancia_cepa = tolerancia[5]
            if porcentaje_lluvia >= tolerancia_cepa:
                contrato = 1
            else:
                contrato = 0

        return contrato
    climas_spot = []
    climas_spot_3 = []
    # Simulación de la etapa 1

    tolerancia_lluvias = tolerancia_cepa(lotes)

    for t in range(50, 150):
        for lote in lotes:
            #El siguiente valor corresponde a el contrato que se utilizara en cada lote
            contrato_lote = tipo_contrato(tolerancia_lluvias,lote)
            lote_id = lote[0]
            lote_cod = lote[1]
            lote_numero = lote[2]
            lote_tipo_uva = lote[3]
            lote_toneladas = lote[4]
            lote_dia_optimo = lote[5]
            lote_prob_seco_lluvia = lote[6]
            lote_prob_lluvia_lluvia = lote[7]
            lote_precio_usd = lote[8]
            lote_variacion = lote[9]
            lote_prob_lluvia = lote[10]
            if t == lote_dia_optimo - 8:
                lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
                lista = lote
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lista.append(calidad_lote)
                lista.append(contrato_lote)
                lista.append(perdida)
                lotes_obtenidos.append(lista)
                if contrato_lote > 0:
                    calidad_lote = calcular_calidad_2(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion[:-2], lote_tipo_uva)
                    if not perdida:
                        climas_spot.append(calidad_lote)
                    elif perdida:
                        climas_spot_3.append(calidad_lote)
                    

    #lote_id = lote[0]
    #lote_cod = lote[1]
    #lote_numero = lote[2]
    #lote_tipo_uva = lote[3]
    #lote_toneladas = lote[4]
    #lote_dia_optimo = lote[5]
    #ote_prob_seco_lluvia = lote[6]
    #lote_prob_lluvia_lluvia = lote[7]
    #lote_precio_usd = lote[8]
    #lote_variacion = lote[9]
    #lote_prob_lluvia = lote[10]
    #calidad lote
    #contrato lote
    #tipo 3(true or false)
    # Modificamos los lotes obtenidos y perdidos con la info relevante, es decir, 
    # sacamos las probabilidades de lluvia y el día óptimo de cosecha, entre otras cosas.
    lista_final = []
    for lote in lotes_obtenidos:
        lista = []
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_costo = lote[8]
        lote_calidad = lote[11]
        lote_contrato = lote[12]
        lote_optimo = lote[5]
        lote_fermentacion = False
        lote_tipo_3 = lote[13]
        lote_prob_lluvia = lote[10]
        lista.append(lote_cod)
        lista.append(lote_numero)
        lista.append(lote_tipo_uva)
        lista.append(lote_toneladas)
        lista.append(lote_costo)
        lista.append(lote_calidad)
        lista.append(lote_contrato)
        lista.append(lote_optimo)
        lista.append(lote_fermentacion)
        lista.append(lote_tipo_3)
        lista.append(lote_prob_lluvia)  
        lista_final.append(lista)


    #lote_cod = lote[1]
    #lote_numero = lote[2]
    #lote_tipo_uva = lote[3]
    #lote_toneladas = lote[4]
    #lote_costo = lote[8]
    #lote_calidad = lote[11]
    #lote_contrato = lote[12]
    #lote_optimo = lote[5]
    #lote_fermentacion = False
    #lote_tipo_3 = lote[13]
    #lote_prob_lluvia = lote[10]

    #Abajo está la recapitulación de la información de la cosecha
    lotes_obtenidos = lista_final
    # Encontrar la info que queramos de esta etapa
    print(f'Calidad máxima: {max(calidades_lotes)}')
    print(f'Calidad mínima: {min(calidades_lotes)}')
    calidad_minima_por_simulacion.append(min(calidades_lotes))
    print('')
    contador = 0
    contador_3 = 0
    for lote in lista_final:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_optimo = lote[7]
        lote_fermentacion = lote[8]
        lote_tipo_3 = lote[9]
        lote_prob_lluvia = lote[10]
        if not lote_tipo_3:
            kilos = lote_toneladas * 1000
            litros = kilos * 0.5
            litros_totales_obtenidos += litros
            costo_lote = kilos*lote[4]
            if lote[6] == 0:
                costo_lote = costo_lote * 0.8
                costos_totales_obtenidos += costo_lote
            elif lote[6] == 1:
                calidad_spot = climas_spot[contador]
                costos_totales_obtenidos += costo_lote*calidad_spot
                contador += 1
        elif lote_tipo_3:
            kilos = lote_toneladas * 1000
            litros = kilos * 0.5
            litros_totales_perdidos += litros
            costo_lote = kilos*lote[4]
            if lote[6] == 0:
                costo_lote = costo_lote * 0.8
                costos_totales_perdidos += costo_lote
            elif lote[6] == 1:
                costo_lote = costo_lote
                calidad_spot_3 = climas_spot_3[contador_3]
                costos_totales_perdidos += costo_lote*calidad_spot_3
    botellas_totales_obtenidas = litros_totales_obtenidos / 0.75
    print(f'Litros totales obtenidos: {round(litros_totales_obtenidos)}')
    litros_totales_perdidos_etapa1 += round(litros_totales_perdidos)
    print(f'Botellas totales obtenidas: {round(botellas_totales_obtenidas)}')
    print(f'Costos totales esperadas USD (considerando contratos spot y foward): {round(costos_totales_obtenidos)}')
    print('')
    botellas_totales_perdidas = litros_totales_perdidos / 0.75
    print(f'Litros totales perdidos: {round(litros_totales_perdidos)}')
    print(f'Botellas totales perdidas: {round(botellas_totales_perdidas)}')
    print(f'Costos totales perdidos USD (compra spot): {round(costos_totales_perdidos)}')
    print('')
    Costos_totales = costos_totales_obtenidos + costos_totales_perdidos
    print(f'Costos totales etapa 1: {round(Costos_totales)}')

    ### ETAPA 2: CALIDAD DE UVA ###
    # Parámetros simulación etapa 2
    costo_recuperado = 0
    contador = 0

    for lote in lista_final:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_optimo = lote[7]
        lote_fermentacion = lote[8]
        lote_tipo_3 = lote[9]
        lote_prob_lluvia = lote[10]
        if lote_tipo_3:
            lote_calidad = 0


        calidad = lote[5]
        cepa = lote[2]
        precio = lote[4]
        kilos = lote[3]*1000
        umbral = umbrales[lote_tipo_uva]
        #Obtenemos costo por lote dependiendo de como ha sido comprado
        ### 1 spot, 0 foward
        if lote_contrato == 0:
            costo = precio * 0.8 * kilos
        elif lote_contrato == 1:
            costo = precio * kilos
        if 0.5 <= lote_calidad < umbral:
            recuperado = costo * 0.3 
        if lote_calidad < 0.5:
            recuperado = costo * 0.05
        else:
            contador += 1
            recuperado = 0
        costo_recuperado += recuperado
    Costos_totales -= costo_recuperado

    print(f'Costos recuperados etapa 2: {round(costo_recuperado)} USD')
    print(f'Costos totales etapa 2, post recuperación: {round(Costos_totales)}')

    ### ETAPA 3: FERMENTACIÓN ###
    # calcula la duracion en fermentación
    def calcular_duracion_vinificacion():
        duracion_aleatoria = random.uniform(7, 9)
        dia_exacto = round(duracion_aleatoria)
        return dia_exacto
    # Parámetros simulación etapa 3
    costo_plantas = {1:[1600, 1840, 1840], 2:[1500, 1725, 1725], 3:[1800, 2070, 2070]}
    uso_plantas = [[0, 0, 0],[0, 0, 0],[0, 0, 0]]
    litros_perdidos = [0, 0, 0, 0, 0, 0]
    costos_perdidos = []
    dias_tanques = []

    def espacio_tanques(uso_plantas):
        espacio_disponible = 0
        for planta in uso_plantas:
            if sum(planta) < 72:
                espacio = 72 - sum(planta)
                espacio_disponible += espacio
        return espacio_disponible

    def litros_uva_perdidos(uva, litros):
        if uva == 'C1':
            litros_perdidos[0] += litros
        elif uva == 'C2':
            litros_perdidos[1] += litros
        elif uva == 'C3':
            litros_perdidos[2] += litros
        elif uva == 'C4':
            litros_perdidos[3] += litros
        elif uva == 'C5':
            litros_perdidos[4] += litros
        elif uva == 'C6':
            litros_perdidos[5] += litros
    def litros_uva_perdidos_tipo_3(uva, litros):
        if uva == 'C1':
            litros_perdidos_tipo_3[0] += litros
        elif uva == 'C2':
            litros_perdidos_tipo_3[1] += litros
        elif uva == 'C3':
            litros_perdidos_tipo_3[2] += litros
        elif uva == 'C4':
            litros_perdidos_tipo_3[3] += litros
        elif uva == 'C5':
            litros_perdidos_tipo_3[4] += litros
        elif uva == 'C6':
            litros_perdidos_tipo_3[5] += litros

    def litros_uva_producidos(uva, litros):
        if uva == 'C1':
            litros_producidos[0] += litros
        elif uva == 'C2':
            litros_producidos[1] += litros
        elif uva == 'C3':
            litros_producidos[2] += litros
        elif uva == 'C4':
            litros_producidos[3] += litros
        elif uva == 'C5':
            litros_producidos[4] += litros
        elif uva == 'C6':
            litros_producidos[5] += litros

    def entrar_tanques(litros, costo, uva):
        botellas = 0
        litros_sobrantes = 0
        condicion = False
        litros_fermentados = 0
        while not condicion:
            if litros > 25000:
                litros -= 25000
                botellas += 1
                litros_fermentados += 25000
            else:
                if litros > 25000 * 0.3:
                    litros_fermentados += litros
                    litros -= litros
                    botellas += 1
                else:
                    litros_sobrantes = litros
                    litros_uva_perdidos(uva, litros_sobrantes)
                    costo_perdido = costo * (litros_sobrantes/0.5)
                    costos_perdidos.append(costo_perdido)
                    condicion = True
        return botellas, litros_fermentados
    def uso_tanques(botellas):
        while botellas > 0:
            if uso_plantas[1][0] < 24:
                uso_plantas[1][0] += 1
                botellas -= 1
            elif uso_plantas[0][0] < 24:
                uso_plantas[0][0] += 1
                botellas -= 1
            elif uso_plantas[1][1] < 24:
                uso_plantas[1][1] += 1
                botellas -= 1
            elif uso_plantas[1][2] < 24:
                uso_plantas[1][2] += 1
                botellas -= 1
            elif uso_plantas[2][0] < 24:
                uso_plantas[2][0] += 1
                botellas -= 1
            elif uso_plantas[0][1] < 24:
                uso_plantas[0][1] += 1
                botellas -= 1
            elif uso_plantas[0][2] < 24:
                uso_plantas[0][2] += 1
                botellas -= 1
            elif uso_plantas[2][1] < 24:
                uso_plantas[2][1] += 1
                botellas -= 1
            elif uso_plantas[2][2] < 24:
                uso_plantas[2][2] += 1
                botellas -= 1
            else:
                break
        return botellas
    def sacar_tanques(botellas):
        while botellas > 0:
            if uso_plantas[1][0] > 0:
                uso_plantas[1][0] -= 1
                botellas -= 1
            elif uso_plantas[0][0] > 0:
                uso_plantas[0][0] -= 1
                botellas -= 1
            elif uso_plantas[1][1] > 0:
                uso_plantas[1][1] -= 1
                botellas -= 1
            elif uso_plantas[1][2] > 0:
                uso_plantas[1][2] -= 1
                botellas -= 1
            elif uso_plantas[2][0] > 0:
                uso_plantas[2][0] -= 1
                botellas -= 1
            elif uso_plantas[0][1] > 0:
                uso_plantas[0][1] -= 1
                botellas -= 1
            elif uso_plantas[0][2] > 0:
                uso_plantas[0][2] -= 1
                botellas -= 1
            elif uso_plantas[2][1] > 0:
                uso_plantas[2][1] -= 1
                botellas -= 1
            elif uso_plantas[2][2] > 0:
                uso_plantas[2][2] -= 1
                botellas -= 1
            else:
                break
    # Más parámetros
    lista_fermentacion = []
    lista_espera = []
    litros_producidos = [0, 0, 0, 0, 0, 0]
    litros_perdidos_tipo_3 = [0, 0, 0, 0, 0, 0]
    for t in range(30, 150):
        for lote in lista_final:
            lote_cod = lote[0]
            lote_numero = lote[1]
            lote_tipo_uva = lote[2]
            lote_toneladas = lote[3]
            lote_costo = lote[4]
            lote_calidad = lote[5]
            lote_contrato = lote[6]
            lote_dia_cosecha = lote[7]
            lote_salida_fermentacion = lote[8]
            lote_tipo_3 = lote[9]
            lote_duracion = calcular_duracion_vinificacion()
            if t == lote_dia_cosecha:
                if lote_tipo_3:
                    # los lotes que tuvieron lluvias tipo 3 se pierden y no entran a fermentación
                    litros_uva_perdidos(lote_tipo_uva, lote_toneladas* 1000 * 0.5)
                    litros_uva_perdidos_tipo_3(lote_tipo_uva, lote_toneladas* 1000 * 0.5)
                    pass
                elif not lote_tipo_3:
                    ### tomar en consideración que cuando dice botellas me refiero a tanques de 
                    ### 25000 litros cada uno.
                    litros_uva = lote_toneladas * 1000 * 0.5
                    botellas, litros_totales_fermentacion = entrar_tanques(litros_uva, lote_costo, lote_tipo_uva)
                    botellas_restantes = uso_tanques(botellas)
                    residuo = litros_totales_fermentacion
                    for i in range(botellas):
                        residuo -= 25000
                    if residuo != 0:
                        residuo = -residuo
                    botellas_a_fermentar = botellas - botellas_restantes
                    litros_finales_fermentados = litros_totales_fermentacion - residuo
                    lista = [t + lote_duracion, botellas_a_fermentar]
                    dias_tanques.append(lista)
                    litros_uva_producidos(lote_tipo_uva, litros_finales_fermentados)
                    if botellas_restantes > 0:
                        print(f'Se perdieron {botellas_restantes} tanques el día {t}')
                        print(botellas_restantes)
                        costo_perdido = lote_costo * ((botellas_restantes*25000)/0.5)
                        print(f'Con el costo de {costo_perdido} USD')
                        print('')
                        litros_uva_perdidos(lote_tipo_uva, botellas_restantes * 25000)
                        costos_perdidos.append(costo_perdido)
                    lote[8] = lote_duracion + t
                    lista_fermentacion.append([lote_tipo_uva, litros_finales_fermentados, t + lote_duracion])
            if lote_salida_fermentacion == t:
                for dias in dias_tanques:
                    if dias[0] == t:
                        botella = dias[1]
                        sacar_tanques(botella)

    # lista para mandar a barricas
    costos_perdidos = round(sum(costos_perdidos))
    print(f'Costos perdidos por no entrar a tanques: {costos_perdidos}')
    Costos_totales += sum([1600, 1840, 1840, 1500, 1725, 1725, 1800, 2070, 2070])
    Costos_totales = round(Costos_totales)
    print(f'Costos totales hasta etapa 3 fermentación: {Costos_totales}')
    print('')
    print('RESULTADOS HASTA FERMENTACIÓN')
    print(f'litros perdidos por cepas (c1 a c6): {litros_perdidos}')
    litros_totales_perdidos_etapa2.append(litros_perdidos)
    print(f'litros producidos por cepas (c1 a c6): {litros_producidos}')
    print(f'litros perdidos por tipo 3 por cepas (c1 a c6): {litros_perdidos_tipo_3}')
    litros_totales_perdidos_etapa2_tipo3.append(litros_perdidos_tipo_3)
    print(f'litros totales perdidos por tipo 3: {sum(litros_perdidos_tipo_3)}')
    print('')
    print(f'litros totales perdidos: {sum(litros_perdidos)}')
    print(f'litros totales producidos: {sum(litros_producidos)}')
    print('')
    botellas_perdidas = [0, 0, 0, 0, 0, 0]
    botellas_producidas = [0, 0, 0, 0, 0, 0]
    for i in range(6):
        botellas_perdidas[i] = litros_perdidos[i] / 0.75
        botellas_producidas[i] = litros_producidos[i] / 0.75
    print(f'botellas totales perdidas: {botellas_perdidas}')
    print(f'botellas totales producidas: {botellas_producidas}')

    ### ETAPA 4: CREACIÓN DE BLENDS Y SATISFACCIÓN DE MERCADO

    ### ETAPA 4: CREACIÓN DE BLENDS Y SATISFACCIÓN DE MERCADO "444444444----------------4----------------------------------4-----------------------------------------------4---------------------------------------------------------4----------------------------------------------4----------"
    ingresos_totales = 0
    import random
    from cmath import inf
    from re import X
    from xml.sax import xmlreader


    data = [
        ("Blend 1", 1, 0.1, 0.2, 0.0, 0.3, 0.0, 0.4),
        ("Blend 2", 1, 0.2, 0.0, 0.2, 0.2, 0.0, 0.4),
        ("Blend 3", 1, 0.5, 0.0, 0.2, 0.0, 0.1, 0.2),
        ("Blend 4", 1, 0.12, 0.15, 0.08, 0.1, 0.1, 0.45),
        ("C1", 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0),
        ("C2", 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0),
        ("C3", 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0),
        ("C4", 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0),
        ("C5", 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0),
        ("C6", 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1)
    ]

    recipe_dict = {}
    precio_mercado = {"demanda_mercado_a": 4, "demanda_mercado_b": 2.8, "demanda_mercado_c": 2.1, "demanda_mercado_d": 1.5}
    precio_cepa = {"C1": 0.27118, "C2": 0.20381, "C3": 0.297487, "C4": 0.135593, "C5": 0.405931, "C6": 0.176233, "Blend 1":0.179, "Blend 2": 0.21134, "Blend 3": 0.27, "Blend 4": 0.22}
    costos_acumulados = {"C1": 0, "C2": 0, "C3": 0, "C4": 0, "C5": 0, "C6": 0}
    nombre_cepa = ["C1","C2","C3","C4","C5","C6"]
    relaciones_mercado_cepa = {
        'demanda_mercado_a': ['C5'],
        'demanda_mercado_b': ['C1', 'C3', 'Blend 2', 'Blend 3'],
        'demanda_mercado_c': ['C2', 'Blend 4'],
        'demanda_mercado_d': ['C4', 'C6', 'Blend 1']
    }      


    ingreso = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d": 0}
    cepa_inicio = {"C1": 0,"C2": 0,"C3": 0,"C4": 0,"C5": 0, "C6": 0, "Blend 1": 0, "Blend 2": 0, "Blend 3": 0, "Blend 4": 0}
    for row in data:
        blend = row[0]
        receta = row[1]
        c_values = row[2:]
        if blend not in recipe_dict:
            recipe_dict[blend] = {}
        recipe_dict[blend][receta] = c_values


    demanda_mercado = [
        ["demanda_mercado_a", 9520000],
        ["demanda_mercado_b", 12693333],
        ["demanda_mercado_c", 11106667],
        ["demanda_mercado_d", 9520000]
    ]

    #Obtenemos cuanto es el maximo de blend que se puede producir segun las cepas que tenemos 
    def procesar_blend(blend):
        c_values = recipe_dict[blend][receta] 
        c_values = list(c_values)
        # Obtener las proporciones de la receta para la mezcla
        # Calcular los factores de escala para cada cepa
        scale_factors = [litro_cepa[i] / (c_values[i]) if c_values[i] != 0 else float('inf') for i in range(0,len(c_values))]
        # Encontrar el factor de escala mínimo
        min_scale_factor = int(min(scale_factors))
        return min_scale_factor


    def ordenar_demanda(demanda_mercado):
        # Utiliza la función sorted para ordenar la lista en orden descendente
        demanda_ordenada = sorted(demanda_mercado, key=lambda x: x[1], reverse=True)
        return demanda_ordenada


    def calcular_utilizacion_cepas(recetas, tipo, cantidad_blend):
        # Suponiendo que receta contiene los porcentajes desde C1 hasta C6
        print(recetas, "Recet")
        utilizacion_cepas = {}
        # Calcular la utilización de cada cepa
        if tipo == "cepa":
            for i in range(6):
                cepa = f"C{i + 1}"  # Generar el nombre de la cepa (C1, C2, ..., C6)
                x = recetas[i]
                utilizacion_cepa = int(cepas_obtenidas[cepa] * x)
                utilizacion_cepas[cepa] = utilizacion_cepa
        else:
            for i in range(6):
                cepa = f"C{i + 1}"  # Generar el nombre de la cepa (C1, C2, ..., C6)
                x = recetas[i]
                utilizacion_cepa = int(cantidad_blend * x)
                utilizacion_cepas[cepa] = utilizacion_cepa


        
        return utilizacion_cepas


    def suma_lista(lista):
        return sum(lista)

    #si se quitan cepas de las cepas guardadas dia a dia se llama a esta funcion para que actualize la cantidad de blends que se pueden producir segun las nuevas cepas que hay
    def actualizar_blends():
        cant_blend_1 = procesar_blend("Blend 1")
        cant_blend_2 = procesar_blend("Blend 2")
        cant_blend_3 = procesar_blend("Blend 3")
        cant_blend_4 = procesar_blend("Blend 4")
        cepas_obtenidas["Blend 1"] = cant_blend_1
        cepas_obtenidas["Blend 2"] = cant_blend_2
        cepas_obtenidas["Blend 3"] = cant_blend_3
        cepas_obtenidas["Blend 4"] = cant_blend_4


    def actualizar_min_botellas_por_mercado():
        global min_botellas_por_mercado
        min_botellas_por_mercado = {
        "Blend 1": demanda_mercado[3][1],  # Asociar Blend 1 con Mercado D, si es necesario
        "Blend 2": demanda_mercado[1][1],  # Asociar Blend 2 con Mercado B, si es necesario
        "Blend 3": demanda_mercado[1][1],  # Asociar Blend 3 con Mercado B, si es necesario
        "Blend 4": demanda_mercado[2][1],  # Asociar Blend 4 con Mercado C, si es necesario
        "C1": demanda_mercado[1][1],       # Asociar C1 con Mercado B, si es necesario
        "C2": demanda_mercado[2][1],       # Asociar C2 con Mercado C, si es necesario
        "C3": demanda_mercado[1][1],       # Asociar C3 con Mercado B, si es necesario
        "C4": demanda_mercado[3][1],       # Asociar C4 con Mercado D, si es necesario
        "C5": demanda_mercado[0][1],       # Asociar C5 con Mercado A, si es necesario
        "C6": demanda_mercado[3][1],       # Asociar C6 con Mercado D, si es necesario
        }      


    def cepas_relacionadas(mercado):
        actualizar_min_botellas_por_mercado()
        return relaciones_mercado_cepa[mercado]


    def cantidades_cepa(cepas):
        cantidad = []
        for cepa in cepas:
            x = cepas_obtenidas[cepa]
            cantidad.append(x)
        return cantidad

    #sum los valores de las cepas de un mercado para luego compararlo con la cantidad de blend que se puede prouducitr
    def sumar_valores_cepas_ce(lista_blend_cepas, lista_valores):
        suma_cepas_c = 0
        for i in range(len(lista_blend_cepas)):
            elemento = lista_blend_cepas[i]
            print(elemento)
            if elemento.startswith('C'):
                suma_cepas_c += lista_valores[i]
        return suma_cepas_c

    #mueve las cepas al principio de la lista para un dia, la lista es de las cepas de un mercado
    def mover_cepas_al_principio(lista_blend_cepas, lista_valores):
        contador = 0
        cepas = []
        otros_elementos = []
        valores_cepas = []
        valores_otros = []

        for i, elemento in enumerate(lista_blend_cepas):
            if elemento.startswith('C'):
                cepas.append(elemento)
                valores_cepas.append(lista_valores[i])
                if lista_valores[i] != 0:
                    contador += 1
            else:
                otros_elementos.append(elemento)
                valores_otros.append(lista_valores[i])


        lista_final = list(cepas) + otros_elementos
        valores_final = list(valores_cepas) + valores_otros

        return lista_final, valores_final, contador

    #encuentra el blend de mayor cantidad a producir para un dia de un mercado especifico
    def encontrar_blend_mayor_cantidad(lista_blend_cepas, lista_valores):
        max_valor = 0
        blend_mayor_cantidad = ""
        for i in range(len(lista_blend_cepas)):
            elemento = lista_blend_cepas[i]
            if elemento.startswith('Blend') and lista_valores[i] > max_valor:
                max_valor = lista_valores[i]
                blend_mayor_cantidad = elemento
        
        return blend_mayor_cantidad, max_valor

    #envia las sobras a cada mercado, si cierto mercado se cumple se acumula pa otro dia para que asi el mercado de mayor demanda tenga harta uva. se asegura tambn de que su demanda se cumpla justo, a exepcion del ukltimo dia t =148 donde se envian todas las sobras a todos los mercados
    def enviar_sobras_a_mercado(c1, c2, c3, c4, c5, c6, litro_cepa, nombre, t):
        print("enviar_sobras_a_mercado")
        resto = 0
        # Asociamos las cepas a sus mercados correspondientes
        #Si la demanda del mercado A es negativa es mejor guardar para blend 4
        if t == 148:
            print("ulti")
            mercado_a = [("C5", c5)]
            mercado_b = [("C1", c1), ("C3", c3)]
            mercado_c = [("C2", c2)]
            mercado_d = [("C4", c4), ("C6", c6)]
            litro_cepa[0] -= cepas_obtenidas["C1"]
            litro_cepa[1] -= cepas_obtenidas["C2"]
            litro_cepa[2] -= cepas_obtenidas["C3"]
            litro_cepa[3] -= cepas_obtenidas["C4"]
            litro_cepa[4] -= cepas_obtenidas["C5"]
            litro_cepa[5] -= cepas_obtenidas["C6"]
            cepa_inicio["C1"] += cepas_obtenidas["C1"]
            cepa_inicio["C2"] += cepas_obtenidas["C2"]
            cepa_inicio["C3"] += cepas_obtenidas["C3"]
            cepa_inicio["C4"] += cepas_obtenidas["C4"]
            cepa_inicio["C5"] += cepas_obtenidas["C5"]
            cepa_inicio["C6"] += cepas_obtenidas["C6"]

            for mercado, cepas in [("mercado_a", mercado_a), ("mercado_b", mercado_b), ("mercado_c", mercado_c), ("mercado_d", mercado_d)]:
                total_enviado = 0
                mensaje = f"El {mercado.replace('_', ' ')} recibió"
                cepas_enviadas = []
                litro_cepa = [0,0,0,0,0,0]

                for cepa, cantidad in cepas:  
                    for demanda_mercado_actual in demanda_mercado:
                        if demanda_mercado_actual[0] == f"demanda_{mercado}":
                            print(cantidad,"cantidad")
                            demanda_mercado_actual[1] -= cantidad

                if total_enviado > 0:
                    mensaje += " " + " y ".join([f"{cantidad} litros de {cepa}" for cepa, cantidad in cepas_enviadas]) 
                    print(mensaje)
            actualizar_blends()
            cepas_obtenidas["C1"] = 0
            cepas_obtenidas["C2"] = 0
            cepas_obtenidas["C3"] = 0
            cepas_obtenidas["C4"] = 0
            cepas_obtenidas["C5"] = 0
            cepas_obtenidas["C6"] = 0
            print(cepas_obtenidas, "====")
                    
            print(demanda_mercado)

        if demanda_mercado[0][1] <= 0:
            resto = 1
            print("DEMANDA MERCADO A NEGATIVA; ACUMULO PA OTRO DIA")

        elif (demanda_mercado[0][1] - c5) <= 0:
            resto = 1
            print("A")
            resto = c5 - demanda_mercado[0][1]
            print(resto)
            litro_cepa[4] -= demanda_mercado[0][1]
            cepa_inicio["C5"] += demanda_mercado[0][1]
            cepas_obtenidas["C5"] -= demanda_mercado[0][1]
            print(demanda_mercado[0][1])
            demanda_mercado[0][1] = 0
            print(demanda_mercado[0][1])

        if demanda_mercado[1][1] <= 0:
            resto = 1
            print("DEMANDA MERCADO B NEGATIVA; ACUMULO PA OTRO DIA")

        elif (demanda_mercado[1][1] - c1 - c3) <= 0:
            resto = 1
            print("B")
            resto = c1 + c3 - demanda_mercado[1][1]
            print(resto)
            print(demanda_mercado[1][1])
            demanda_mercado[1][1] = 0
            print(demanda_mercado[1][1])
            if c1 > 0 and c3 > 0:
                litro_cepa[0] -= int(demanda_mercado[1][1]/2)
                cepa_inicio["C1"] += int(demanda_mercado[1][1]/2)
                litro_cepa[2] -= int(demanda_mercado[1][1]/2)
                cepa_inicio["C3"] += int(demanda_mercado[1][1]/2)
                cepas_obtenidas["C1"] -= int(demanda_mercado[1][1]/2)
                cepas_obtenidas["C3"] -= int(demanda_mercado[1][1]/2)
            elif c1 > 0 and c3 == 0:
                litro_cepa[0] -= int(demanda_mercado[1][1])
                cepa_inicio["C1"] += int(demanda_mercado[1][1])
                cepas_obtenidas["C1"] -= int(demanda_mercado[1][1])
            elif c1 == 0 and c3 > 0:
                litro_cepa[2] -= int(demanda_mercado[1][1])
                cepa_inicio["C3"] += int(demanda_mercado[1][1])
                cepas_obtenidas["C3"] -= int(demanda_mercado[1][1])

        
        if demanda_mercado[2][1] <= 0:
            resto = 1
            print("DEMANDA MERCADO D NEGATIVA; ACUMULO PA OTRO DIA")

        elif (demanda_mercado[2][1] - c2) <= 0:
            resto = 1
            print("C")
            resto = c2 - demanda_mercado[2][1]
            print(resto)
            litro_cepa[1] -= demanda_mercado[2][1]
            cepa_inicio["C2"] += demanda_mercado[2][1]
            cepas_obtenidas["C2"] -= int(demanda_mercado[2][1])
            print(demanda_mercado[2][1])
            demanda_mercado[2][1] = 0
            print(demanda_mercado[2][1])
        
        if demanda_mercado[3][1] <= 0:
            resto = 1
            print("DEMANDA MERCADO D NEGATIVA; ACUMULO PA OTRO DIA")

        elif (demanda_mercado[3][1] - c4 - c6) <= 0:
            resto = 1
            print("D")
            resto = c4 + c6 - demanda_mercado[3][1]
            print(resto)
            litro_cepa[3] -= int(demanda_mercado[3][1]/2)
            cepa_inicio["C4"] += int(demanda_mercado[3][1]/2)
            print(demanda_mercado[3][1])
            demanda_mercado[3][1] = 0
            print(demanda_mercado[3][1])
            litro_cepa[5] -= int(demanda_mercado[3][1]/2)
            cepa_inicio["C6"] += int(demanda_mercado[3][1]/2)
            if c4 > 0 and c6 > 0:
                litro_cepa[3] -= int(demanda_mercado[3][1]/2)
                cepa_inicio["C4"] += int(demanda_mercado[3][1]/2)
                litro_cepa[5] -= int(demanda_mercado[3][1]/2)
                cepa_inicio["C6"] += int(demanda_mercado[3][1]/2)
                cepas_obtenidas["C4"] -= int(demanda_mercado[3][1]/2)
                cepas_obtenidas["C6"] -= int(demanda_mercado[3][1]/2)
            elif c4 > 0 and c6 == 0:
                litro_cepa[3] -= int(demanda_mercado[3][1])
                cepa_inicio["C4"] += int(demanda_mercado[3][1])
                cepas_obtenidas["C4"] -= int(demanda_mercado[3][1])
            elif c4 == 0 and c6 > 0:
                litro_cepa[5] -= int(demanda_mercado[3][1])
                cepa_inicio["C6"] += int(demanda_mercado[3][1])
                cepas_obtenidas["C6"] -= int(demanda_mercado[3][1])
           
        elif resto == 0:
            mercado_a = [("C5", c5)]
            mercado_b = [("C1", c1), ("C3", c3)]
            mercado_c = [("C2", c2)]
            mercado_d = [("C4", c4), ("C6", c6)]
            litro_cepa[0] -= cepas_obtenidas["C1"]
            litro_cepa[1] -= cepas_obtenidas["C2"]
            litro_cepa[2] -= cepas_obtenidas["C3"]
            litro_cepa[3] -= cepas_obtenidas["C4"]
            litro_cepa[4] -= cepas_obtenidas["C5"]
            litro_cepa[5] -= cepas_obtenidas["C6"]
            cepa_inicio["C1"] += cepas_obtenidas["C1"]
            cepa_inicio["C2"] += cepas_obtenidas["C2"]
            cepa_inicio["C3"] += cepas_obtenidas["C3"]
            cepa_inicio["C4"] += cepas_obtenidas["C4"]
            cepa_inicio["C5"] += cepas_obtenidas["C5"]
            cepa_inicio["C6"] += cepas_obtenidas["C6"]

            for mercado, cepas in [("mercado_a", mercado_a), ("mercado_b", mercado_b), ("mercado_c", mercado_c), ("mercado_d", mercado_d)]:
                total_enviado = 0
                mensaje = f"El {mercado.replace('_', ' ')} recibió"
                cepas_enviadas = []
                litro_cepa = [0,0,0,0,0,0]

                for cepa, cantidad in cepas:  
                    for demanda_mercado_actual in demanda_mercado:
                        if demanda_mercado_actual[0] == f"demanda_{mercado}":
                            print(cantidad,"cantidad")
                            demanda_mercado_actual[1] -= cantidad

                if total_enviado > 0:
                    mensaje += " " + " y ".join([f"{cantidad} litros de {cepa}" for cepa, cantidad in cepas_enviadas]) 
                    print(mensaje)
            actualizar_blends()
            cepas_obtenidas["C1"] = 0
            cepas_obtenidas["C2"] = 0
            cepas_obtenidas["C3"] = 0
            cepas_obtenidas["C4"] = 0
            cepas_obtenidas["C5"] = 0
            cepas_obtenidas["C6"] = 0
            print(cepas_obtenidas, "====")
                    
            print(demanda_mercado)




    def sumar_blend(blend_mayor):
        cepa_inicio[blend_mayor] += cepa_inicio[blend_mayor]

    #resto la receta de la cantidad de uvas a ocupar y actualizo el mercado
    def procesar_receta(primer_nombre, recetas, tipo, nombre, cantidad_blend):
        utilizacion_cepas = calcular_utilizacion_cepas(recetas, tipo, cantidad_blend)
        print(utilizacion_cepas,"UUU")
        if tipo == "cepa":
            if nombre == "C1":
                if demanda_mercado[1][1] - utilizacion_cepas["C1"] < 0:
                    cepa_inicio["C1"] += demanda_mercado[1][1]
                    cepas_obtenidas["C1"] -= demanda_mercado[1][1]
                    demanda_mercado[1][1] = 0

                else:
                    print("C1")
                    cepa_inicio["C1"] += utilizacion_cepas["C1"]
                    demanda_mercado[1][1] -= utilizacion_cepas["C1"]
                    cepas_obtenidas["C1"] = 0
            
            elif nombre == "C2":
                if demanda_mercado[2][1] - utilizacion_cepas["C2"] < 0:
                    cepa_inicio["C2"] += demanda_mercado[2][1]
                    cepas_obtenidas["C2"] -= demanda_mercado[2][1]
                    demanda_mercado[2][1] = 0
                
                else:
                    print("C2")
                    litro_cepa[1] -= utilizacion_cepas["C2"]
                    cepa_inicio["C2"] += utilizacion_cepas["C2"]
                    demanda_mercado[2][1] -= utilizacion_cepas["C2"]
                    cepas_obtenidas["C2"] = 0
                    

            elif nombre == "C3":
                if demanda_mercado[1][1] - utilizacion_cepas["C3"] < 0:
                    cepa_inicio["C3"] += demanda_mercado[1][1]
                    cepas_obtenidas["C3"] -= demanda_mercado[1][1]
                    demanda_mercado[1][1] = 0
                
                else:
                    print("C3")
                    cepa_inicio["C3"] += utilizacion_cepas["C3"]
                    demanda_mercado[1][1] -= utilizacion_cepas["C3"]
                    cepas_obtenidas["C3"] = 0

            elif nombre == "C4":
                if demanda_mercado[3][1] - utilizacion_cepas["C4"] < 0:
                    cepa_inicio["C4"] += demanda_mercado[3][1]
                    cepas_obtenidas["C4"] -= demanda_mercado[3][1]
                    demanda_mercado[3][1] = 0
                
                else:
                    print("C4")
                    cepa_inicio["C4"] += utilizacion_cepas["C4"]
                    demanda_mercado[3][1] -= utilizacion_cepas["C4"]
                    cepas_obtenidas["C4"] = 0


            elif nombre == "C5":
                if demanda_mercado[0][1] - utilizacion_cepas["C5"] < 0:
                    cepa_inicio["C5"] += demanda_mercado[0][1]
                    cepas_obtenidas["C5"] -= demanda_mercado[0][1]
                    demanda_mercado[0][1] = 0
                
                else:
                    print("C5")
                    cepa_inicio["C5"] += utilizacion_cepas["C5"]
                    demanda_mercado[0][1] -= utilizacion_cepas["C5"]
                    cepas_obtenidas["C5"] = 0


            elif nombre == "C6":
                if demanda_mercado[3][1] - utilizacion_cepas["C6"] < 0:
                    cepa_inicio["C6"] += demanda_mercado[3][1]
                    cepas_obtenidas["C6"] -= demanda_mercado[3][1]
                    demanda_mercado[3][1] = 0
                
                else:
                    print("C6")
                    cepa_inicio["C6"] += utilizacion_cepas["C6"]
                    demanda_mercado[3][1] -= utilizacion_cepas["C6"]
                    cepas_obtenidas["C6"] = 0


        elif tipo == "Blend":
            if nombre == "Blend 1":
                if demanda_mercado[3][1] - cepas_obtenidas["Blend 1"] <= 0:
                    cepa_inicio["Blend 1"] += demanda_mercado[3][1]
                    utilizacion_if_blend_casi_cero = calcular_utilizacion_cepas(recetas, tipo, demanda_mercado[3][1])
    
                    demanda_mercado[3][1] = 0
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_if_blend_casi_cero[cepa]

                else:
                    cepa_inicio["Blend 1"] += cepas_obtenidas["Blend 1"]
                    
                    demanda_mercado[3][1] -= cepas_obtenidas["Blend 1"]
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_cepas[cepa]

            elif nombre == "Blend 2":
                if demanda_mercado[1][1] - cepas_obtenidas["Blend 2"] <= 0:
                    cepa_inicio["Blend 2"] += demanda_mercado[1][1]
                    utilizacion_if_blend_casi_cero = calcular_utilizacion_cepas(recetas, tipo, demanda_mercado[1][1])

                    demanda_mercado[1][1] = 0
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_if_blend_casi_cero[cepa]
                    
                
                else:
                    cepa_inicio["Blend 2"] += cepas_obtenidas["Blend 2"]

                    demanda_mercado[1][1] -= cepas_obtenidas["Blend 2"]
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_cepas[cepa]

            elif nombre == "Blend 3":
                if demanda_mercado[1][1] - cepas_obtenidas["Blend 3"] <= 0:
                    cepa_inicio["Blend 3"] += demanda_mercado[1][1]
                    utilizacion_if_blend_casi_cero = calcular_utilizacion_cepas(recetas, tipo, demanda_mercado[1][1])

                    demanda_mercado[1][1] = 0
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_if_blend_casi_cero[cepa]
                    
                
                else:
                    cepa_inicio["Blend 3"] += cepas_obtenidas["Blend 3"]

                    demanda_mercado[1][1] -= cepas_obtenidas["Blend 3"]
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_cepas[cepa]

            elif nombre == "Blend 4":
                if demanda_mercado[2][1] - cepas_obtenidas["Blend 4"] <= 0:
                    cepa_inicio["Blend 4"] += demanda_mercado[2][1]
                    utilizacion_if_blend_casi_cero = calcular_utilizacion_cepas(recetas, tipo, demanda_mercado[2][1])

                    demanda_mercado[2][1] = 0
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_if_blend_casi_cero[cepa]
                    
                
                else:
                    cepa_inicio["Blend 4"] += cepas_obtenidas["Blend 4"]
                    demanda_mercado[2][1] -= cepas_obtenidas["Blend 4"]
                    for cepa in nombre_cepa:
                        cepas_obtenidas[cepa] -= utilizacion_cepas[cepa]
        
            





    # Restricciones mínimas por mercado (ajustadas)
    min_botellas_por_mercado = {
        "Blend 1": demanda_mercado[3][1],  # Asociar Blend 1 con Mercado D, si es necesario
        "Blend 2": demanda_mercado[1][1],  # Asociar Blend 2 con Mercado B, si es necesario
        "Blend 3": demanda_mercado[1][1],  # Asociar Blend 3 con Mercado B, si es necesario
        "Blend 4": demanda_mercado[2][1],  # Asociar Blend 4 con Mercado C, si es necesario
        "C1": demanda_mercado[1][1],       # Asociar C1 con Mercado B, si es necesario
        "C2": demanda_mercado[2][1],       # Asociar C2 con Mercado C, si es necesario
        "C3": demanda_mercado[1][1],       # Asociar C3 con Mercado B, si es necesario
        "C4": demanda_mercado[3][1],       # Asociar C4 con Mercado D, si es necesario
        "C5": demanda_mercado[0][1],       # Asociar C5 con Mercado A, si es necesario
        "C6": demanda_mercado[3][1],       # Asociar C6 con Mercado D, si es necesario
    }


    def suma_demanda():
        suma = 0
        for i in demanda_mercado:
            suma += i[1]
        return suma


    # Inicializamos las variables para los litros en cada mercado

    C1_fer = 0
    C2_fer = 0
    C3_fer = 0
    C4_fer = 0
    C5_fer = 0
    C6_fer = 0
    for i in lista_fermentacion:
        if i[0] == "C1":
            C1_fer += i[1]
        elif i[0] == "C2":
            C2_fer += i[1]
        elif i[0] == "C3":
            C3_fer += i[1]
        elif i[0] == "C4":
            C4_fer += i[1]
        elif i[0] == "C5":
            C5_fer += i[1]
        elif i[0] == "C6":
            C6_fer += i[1]

    



    # Inicializamos las variables para los litros sobrantes por mercado
    sobrantes_a = 0
    sobrantes_b = 0
    sobrantes_c = 0
    sobrantes_d = 0

    cepas_obtenidas = {
            'C1': 0,
            'C2': 0,
            'C3': 0,
            'C4': 0,
            'C5': 0,
            'C6': 0,
        }
    p = 0
    litro_cepa = [0 , 0, 0, 0, 0, 0]
    contador_dias_blend4 = 0
    for t in range(65, 150):
        cant = 0
        cepa_c1 = 0
        cepa_c2 = 0
        cepa_c3 = 0
        cepa_c4 = 0
        cepa_c5 = 0
        cepa_c6 = 0
        print(f"------{t}--------------------")
        for lote in lista_fermentacion:
            if lote[2] == t:
                if lote[0] == "C1":
                    cepa_c1 += lote[1] 
                    cant += 1
        
                if lote[0] == "C2":
                    cepa_c2 += lote[1]
                    cant += 1
                
                if lote[0] == "C3":
                    cepa_c3 += lote[1]
                    cant += 1
                
                if lote[0] == "C4":
                    cepa_c4 += lote[1]
                    cant += 1 
                
                if lote[0] == "C5":
                    cepa_c5 += lote[1]
                    cant += 1
                
                if lote[0] == "C6":
                    cepa_c6 += lote[1] 
                    cant += 1 
        if t == 148:
            print("ULTIMO")
            print(cepas_obtenidas, "cepas_obtenidas")
            enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                          
        if cant == 0:
            print("NO HAY CEPAS ESTE DIA")
            pass
        else:
            print(cepa_c1, cepa_c2, cepa_c3, cepa_c4, cepa_c5, cepa_c6,"CEPAS")
            cepas_obtenidas['C1'] += cepa_c1
            cepas_obtenidas['C2'] += cepa_c2    
            cepas_obtenidas['C3'] += cepa_c3
            cepas_obtenidas['C4'] += cepa_c4
            cepas_obtenidas['C5'] += cepa_c5
            cepas_obtenidas['C6'] += cepa_c6
            litro_cepa[0] += cepa_c1
            litro_cepa[1] += cepa_c2
            litro_cepa[2] += cepa_c3
            litro_cepa[3] += cepa_c4
            litro_cepa[4] += cepa_c5
            litro_cepa[5] += cepa_c6
            cant_blend_1 = procesar_blend("Blend 1")
            cant_blend_2 = procesar_blend("Blend 2")
            cant_blend_3 = procesar_blend("Blend 3")
            cant_blend_4 = procesar_blend("Blend 4")
            cepas_obtenidas["Blend 1"] = cant_blend_1
            cepas_obtenidas["Blend 2"] = cant_blend_2
            cepas_obtenidas["Blend 3"] = cant_blend_3
            cepas_obtenidas["Blend 4"] = cant_blend_4
            print(cepas_obtenidas,"inicio")
            existe_demanda_positiva = 0
            contar = 0
            for i in demanda_mercado:
                if i[1] > 0:
                    existe_demanda_positiva = 1
                    contar += 1

            if existe_demanda_positiva == 1:
            # Inicializamos las variables para las cepas utilizadas por día
                demanda_mercado_ordenada = ordenar_demanda(demanda_mercado)
                mercado_mayor_demanda = demanda_mercado_ordenada[0][1]
                nombre_mercado_mayor_demanda = demanda_mercado_ordenada[0][0]
                    # Accede a las cepas relacionadas a ese mercado desde el diccionario min_botellas_por_mercado
            elif existe_demanda_positiva == 0:
                pos = random.randint(0, 3)
                mercado_mayor_demanda = demanda_mercado_ordenada[pos][1]
                nombre_mercado_mayor_demanda = demanda_mercado_ordenada[pos][0]
            print(mercado_mayor_demanda,"demanda inicio dia", nombre_mercado_mayor_demanda)
            cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
            lista_valores = cantidades_cepa(cepas)
            print(cepas)
            print(lista_valores)
            
            suma = suma_lista(lista_valores)
            if nombre_mercado_mayor_demanda == "demanda_mercado_c" and suma != 0:
                lista_combinada = list(zip(cepas, lista_valores))
                lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                nombres_ordenados, lista_valores = zip(*lista_ordenada)
                nombre_ordenados = list(nombres_ordenados)
                lista_valores = list(lista_valores)
                x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombre_ordenados, lista_valores) #max_valor == Blend4
                print(blend_mayor)
                if max_valor < x and cepa_inicio["Blend 4"] <= 1300000:

                    contador_dias_blend4 += 1
                    p = "C"
                    if contador_dias_blend4 == 9:
                        contador_dias_blend4 = 0
                        p = 0
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'],litro_cepa, nombre_mercado_mayor_demanda, t)
                        pass
                elif max_valor >= x or cepa_inicio["Blend 4"] >= 1300000:
                    
                    contador_dias_blend4 = 0
                    p = 0
                    primer_nombre = "Blend 4"
                      # Obtener el primer nombre de la lista
                    # Buscar las recetas correspondientes en data
                    recetas = []
                    for item in data:
                        if item[0] == primer_nombre:
                            recetas = item[2:]  # Excluir el primer elemento (nombre)
                            break
                    print(recetas, "receta")
                    print(lista_valores)
                    procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre, max_valor)
                    print(lista_valores[0])
                    litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                    actualizar_blends()
                    print(cepas_obtenidas, "cepas_obtenidas")
                    

                    
                            
            else:
                if suma == 0: #entonces pasar al siguiente mercado dado que no existe cepas para producir en este mercado
                    mercado_mayor_demanda = demanda_mercado_ordenada[1][1]
                    if mercado_mayor_demanda <= 0:
                        print("SIGUIENTE DIA")
                        pass
                    else:
                        print(mercado_mayor_demanda)
                        nombre_mercado_mayor_demanda = demanda_mercado_ordenada[1][0]
                        print(nombre_mercado_mayor_demanda)
                        cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                        print(cepas)
                        lista_valores = cantidades_cepa(cepas)
                        print(lista_valores)
                        suma = suma_lista(lista_valores)
                        if nombre_mercado_mayor_demanda == "demanda_mercado_c" and suma != 0:
                            
                            lista_combinada = list(zip(cepas, lista_valores))
                            lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                            nombres_ordenados, lista_valores = zip(*lista_ordenada)
                            nombre_ordenados = list(nombres_ordenados)
                            lista_valores = list(lista_valores)
                            x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                            blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                            if max_valor < x and cepa_inicio["Blend 4"] <= 1300000:
                                contador_dias_blend4 += 1
                                p = "C"
                                if contador_dias_blend4 == 9:
                                    contador_dias_blend4 = 0
                                    p = 0
                                    enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'],litro_cepa, nombre_mercado_mayor_demanda, t)
                                pass
                            elif max_valor >= x or cepa_inicio["Blend 4"] >= 1300000:
                                contador_dias_blend4 = 0
                                p = 0
                                primer_nombre = "Blend 4"
                                 # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = []
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                print(receta,"receta")
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre, max_valor)
                                print(lista_valores[0])
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                                

                        if suma == 0:
                            mercado_mayor_demanda = demanda_mercado_ordenada[2][1]
                            if mercado_mayor_demanda <= 0:
                                print("SIGUIENTE DIA")
                                pass
                            else:
                                print(mercado_mayor_demanda)
                                nombre_mercado_mayor_demanda = demanda_mercado_ordenada[2][0]
                                print(nombre_mercado_mayor_demanda)
                                cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                                print(cepas)
                                lista_valores = cantidades_cepa(cepas)
                                print(lista_valores)
                                suma = suma_lista(lista_valores)
                                if suma == 0:
                                    mercado_mayor_demanda = demanda_mercado_ordenada[3][1]
                                    print(mercado_mayor_demanda)
                                    nombre_mercado_mayor_demanda = demanda_mercado_ordenada[3][0]
                                    print(nombre_mercado_mayor_demanda)
                                    cepas = cepas_relacionadas(nombre_mercado_mayor_demanda)
                                    print(cepas)
                                    lista_valores = cantidades_cepa(cepas)
                                    print(lista_valores)
                                    lista_combinada = list(zip(cepas, lista_valores))
                                    lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                                    nombres_ordenados, lista_valores = zip(*lista_ordenada)
                                    nombre_ordenados = list(nombres_ordenados)
                                    lista_valores = list(lista_valores)
                                    x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                                    blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                                    if max_valor >= x:
                                        primer_nombre = blend_mayor
                                        print(primer_nombre)  # Obtener el primer nombre de la lista
                                        # Buscar las recetas correspondientes en data
                                        recetas = None
                                        for item in data:
                                            if item[0] == primer_nombre:
                                                recetas = item[2:]  # Excluir el primer elemento (nombre)
                                                break
                                        procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", primer_nombre, max_valor)
                                        print(lista_valores[0])
                                        litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                        actualizar_blends()
                                        print(cepas_obtenidas, "cepas_obtenidas")
                                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                                        
                                    elif p == 0:
                                        lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                                        print(lista_valores, lista_blend_cepas)
                                        for i in range(0, contador):
                                            primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                            # Buscar las recetas correspondientes en data
                                            recetas = None
                                            for item in data:
                                                if item[0] == primer_nombre:
                                                    recetas = item[2:]  # Excluir el primer elemento (nombre)
                                                    break
                                            print(lista_valores[i])
                                            procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i], max_valor)
                                            litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                            actualizar_blends()
                                            print(cepas_obtenidas, "cepas_obtenidas")
                                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                                       
                                
                                else:
                                    
                                    lista_combinada = list(zip(cepas, lista_valores))
                                    lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                                    nombres_ordenados, lista_valores = zip(*lista_ordenada)
                                    nombre_ordenados = list(nombres_ordenados)
                                    lista_valores = list(lista_valores)
                                    x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                                    blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                                    if max_valor >= x:
                                        primer_nombre = nombres_ordenados[0]
                                        print(primer_nombre)  # Obtener el primer nombre de la lista
                                        # Buscar las recetas correspondientes en data
                                        recetas = None
                                        for item in data:
                                            if item[0] == primer_nombre:
                                                recetas = item[2:]  # Excluir el primer elemento (nombre)
                                                break
                                        procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor, max_valor)
                                        print(lista_valores[0])
                                        litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                        actualizar_blends()
                                        print(cepas_obtenidas, "cepas_obtenidas")
                                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                                    elif p == 0:
                                        lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                                        print(lista_valores, lista_blend_cepas)
                                        for i in range(0, contador): 
                                            primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                            # Buscar las recetas correspondientes en data
                                            recetas = None
                                            for item in data:
                                                if item[0] == primer_nombre:
                                                    recetas = item[2:]  # Excluir el primer elemento (nombre)
                                                    break
                                            print(lista_valores[i])
                                            procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i], max_valor)
                                            litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                            actualizar_blends()
                                            print(cepas_obtenidas, "cepas_obtenidas")
                                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                                        

                        elif p == 0:
                           
                            lista_combinada = list(zip(cepas, lista_valores))
                            lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                            nombres_ordenados, lista_valores = zip(*lista_ordenada)
                            nombre_ordenados = list(nombres_ordenados)
                            lista_valores = list(lista_valores)
                            x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                            blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                            if max_valor >= x:
                                lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                                nombres_ordenados, lista_valores = zip(*lista_ordenada)
                                primer_nombre = nombres_ordenados[0]  # Obtener el primer nombre de la lista
                                # Buscar las recetas correspondientes en data
                                recetas = None
                                for item in data:
                                    if item[0] == primer_nombre:
                                        recetas = item[2:]  # Excluir el primer elemento (nombre)
                                        break
                                print(lista_valores[0])
                                procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor, max_valor)
                                litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                actualizar_blends()
                                print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                               

                            else:
                                lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                                for i in range(0, contador): 
                                    primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                                    # Buscar las recetas correspondientes en data
                                    recetas = None
                                    for item in data:
                                        if item[0] == primer_nombre:
                                            recetas = item[2:]  # Excluir el primer elemento (nombre)
                                            break
                                    print(lista_valores[i])
                                    procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i], max_valor)
                                    litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                                    actualizar_blends()
                                    print(cepas_obtenidas, "cepas_obtenidas")
                                enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                               

                else:
                    lista_combinada = list(zip(cepas, lista_valores))
                    lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                    nombres_ordenados, lista_valores = zip(*lista_ordenada)
                    nombre_ordenados = list(nombres_ordenados)
                    lista_valores = list(lista_valores)
                    x = sumar_valores_cepas_ce(nombres_ordenados, lista_valores) #x == C2
                    blend_mayor, max_valor = encontrar_blend_mayor_cantidad(nombres_ordenados, lista_valores) #max_valor == Blend4
                    if max_valor >= x:
                        lista_ordenada = sorted(lista_combinada, key=lambda x: x[1], reverse=True)
                        nombres_ordenados, lista_valores = zip(*lista_ordenada)
                        primer_nombre = nombres_ordenados[0]  # Obtener el primer nombre de la lista
                        # Buscar las recetas correspondientes en data
                        recetas = None
                        for item in data:
                            if item[0] == primer_nombre:
                                recetas = item[2:]  # Excluir el primer elemento (nombre)
                                break
                        print(lista_valores[0])
                        procesar_receta(nombre_mercado_mayor_demanda, recetas, "Blend", blend_mayor, max_valor)
                        litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                        actualizar_blends()
                        print(cepas_obtenidas, "cepas_obtenidas")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                    else:
                        lista_blend_cepas, lista_valores, contador = mover_cepas_al_principio(nombres_ordenados, lista_valores)
                        for i in range(0, contador): 
                            primer_nombre = lista_blend_cepas[i]  # Obtener el primer nombre de la lista
                            # Buscar las recetas correspondientes en data
                            recetas = None
                            for item in data:
                                if item[0] == primer_nombre:
                                    recetas = item[2:]  # Excluir el primer elemento (nombre)
                                    break
                            print(lista_valores[i])
                            procesar_receta(nombre_mercado_mayor_demanda, recetas, "cepa", lista_blend_cepas[i], max_valor)
                            litro_cepa = [cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6']]
                            actualizar_blends()
                            print(cepas_obtenidas, "cepas_obtenidas")
                        enviar_sobras_a_mercado(cepas_obtenidas['C1'], cepas_obtenidas['C2'], cepas_obtenidas['C3'], cepas_obtenidas['C4'], cepas_obtenidas['C5'], cepas_obtenidas['C6'], litro_cepa, nombre_mercado_mayor_demanda, t)
                        

            print(cepa_inicio, "cepa_producida")
            # Ordenar la lista combinada por los valores en orden descendente
            # Desempacar las listas ordenadas
        print(demanda_mercado)       

        

        # Verifica si se encontró el mercado y, si es así, imprime su demanda
    u = 0
    for i in demanda_mercado:
        if i[1] != 0:
            u += 1
    if u != 0:
        g += 1
    

    costos_mercado = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d": 0}

    mercado_final = {"demanda_mercado_a": 0, "demanda_mercado_b": 0, "demanda_mercado_c": 0, "demanda_mercado_d":0}
    print(demanda_mercado, "Demanda_mercado")
    # Calcular costos y sumarlos a los diccionarios correspondientes
    costos_acumulados
    x = ingreso["demanda_mercado_a"] = cepa_inicio["C5"]/0.75*precio_mercado["demanda_mercado_a"]
    y = ingreso["demanda_mercado_b"] = cepa_inicio["C1"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["C3"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["Blend 2"]/0.75*precio_mercado["demanda_mercado_b"] + cepa_inicio["Blend 3"]/0.75*precio_mercado["demanda_mercado_b"]
    z =ingreso["demanda_mercado_c"] = cepa_inicio["C2"]/0.75*precio_mercado["demanda_mercado_c"] + cepa_inicio["Blend 4"]/0.75*precio_mercado["demanda_mercado_c"]
    w = ingreso["demanda_mercado_d"] = cepa_inicio["C4"]/0.75*precio_mercado["demanda_mercado_d"] + cepa_inicio["C6"]/0.75*precio_mercado["demanda_mercado_d"] + cepa_inicio["Blend 1"]/0.75*precio_mercado["demanda_mercado_d"]
    print(ingreso, "INGRESOS ; SIN COSTOS RESTADOS")

    ca = costos_mercado["demanda_mercado_a"] = (cepa_inicio["C5"]/0.75)*2.5 
    cb = costos_mercado["demanda_mercado_b"] = (cepa_inicio["C1"]/0.75)*2 + (cepa_inicio["C3"]/0.75)*2 + (cepa_inicio["Blend 2"]/0.75)*2  + (cepa_inicio["Blend 3"]/0.75)*2
    cc = costos_mercado["demanda_mercado_c"] = (cepa_inicio["C2"]/0.75)*1.45 + (cepa_inicio["Blend 4"]/0.75)*1.45
    cd = costos_mercado["demanda_mercado_d"] = (cepa_inicio["C4"]/0.75)*1.17 + (cepa_inicio["C6"]/0.75)*1.17 + (cepa_inicio["Blend 1"]/0.75)*1.17

    print(C1_fer, C2_fer, C3_fer, C4_fer, C5_fer, C6_fer, "Cantidad de cepa de fermentacion")
    C1_fermentacion_suma += C1_fer
    C2_fermentacion_suma += C2_fer
    C3_fermentacion_suma += C3_fer
    C4_fermentacion_suma += C4_fer
    C5_fermentacion_suma += C5_fer
    C6_fermentacion_suma += C6_fer
    mercado_final["demanda_mercado_a"] = cepa_inicio["C5"]
    mercado_final["demanda_mercado_b"] = cepa_inicio["C1"] + cepa_inicio["C3"] + cepa_inicio["Blend 2"] + cepa_inicio["Blend 3"]
    mercado_final["demanda_mercado_c"] = cepa_inicio["C2"] + cepa_inicio["Blend 4"]
    mercado_final["demanda_mercado_d"] = cepa_inicio["C4"] + cepa_inicio["C6"] + cepa_inicio["Blend 1"]
    lista_mercados = [mercado_final["demanda_mercado_a"], mercado_final["demanda_mercado_b"], mercado_final["demanda_mercado_c"], mercado_final["demanda_mercado_d"]]
    demandas_por_mercado.append(lista_mercados)
    print("CANTIDAD PRODUCIDA POR CEPA Y BLENDS")
    print(cepa_inicio,"producido")
    print("DEMANDA QUE SE LOGRO LLENAR POR MERCADO")
    print(mercado_final,"mercado producido")
    print("INGRESOS POR MERCADO (precio minimo)")
    print(ingreso,"Ing")
    print("COSTOS POR MERCADO (solo costo de cepa)")
    print(costos_mercado, "costo_mercado")
    COSTO_TOTAL = costos_mercado["demanda_mercado_a"] + costos_mercado["demanda_mercado_b"] + costos_mercado["demanda_mercado_c"] + costos_mercado["demanda_mercado_d"]
    print("COSTO TOTAL:", COSTO_TOTAL)
    INGRESO_TOTAL = ingreso["demanda_mercado_a"] + ingreso["demanda_mercado_b"] +  ingreso["demanda_mercado_c"] + ingreso["demanda_mercado_d"]
    print("INGRESO TOTAL:", INGRESO_TOTAL)
    Costos_totales += COSTO_TOTAL
    ingresos_totales += INGRESO_TOTAL
    Costos_totales = round(Costos_totales)
    ingresos_totales = round(ingresos_totales)
    print(Costos_totales, "Costos_totales")
    print(ingresos_totales, "ingreso_total")
    utilidad = round(ingresos_totales - Costos_totales)
    print(utilidad, "utilidad")
    ingresos_totales_finales.append(ingresos_totales)
    costos_totales_finales.append(Costos_totales)
    utilidad_total_final.append(utilidad)
    f = [cepa_inicio["C1"],cepa_inicio["C2"],cepa_inicio["C3"],cepa_inicio["C4"],cepa_inicio["C5"],cepa_inicio["C6"],cepa_inicio["Blend 1"],cepa_inicio["Blend 2"],cepa_inicio["Blend 3"],cepa_inicio["Blend 4"]]
    cepa_inicio_total.append(f)
    #Obtenemos las veces que el mercado no se satisface para cada mercado, la suma de sus resultados y la suma de las veces en las que es positiva para asi obtener un promedio. si es positiva es porque falto botellas para cumplir mercado
    for i in range (0, len(demanda_mercado)):
        if demanda_mercado[i][0] == "demanda_mercado_a":
            if demanda_mercado[i][1] != 0:
                sobra_a += demanda_mercado[i][1]
            if demanda_mercado[i][1] > 0:
                veces_mercado_a_no_cumple += 1
                suma_resto_mercado_a_positiva_solamente += demanda_mercado[i][1]
        elif demanda_mercado[i][0] == "demanda_mercado_b":
            if demanda_mercado[i][1] != 0:
                sobra_b += demanda_mercado[i][1]
                
            if demanda_mercado[i][1] > 0:
                veces_mercado_b_no_cumple += 1
                suma_resto_mercado_b_positiva_solamente += demanda_mercado[i][1]
        elif demanda_mercado[i][0] == "demanda_mercado_c":
            if demanda_mercado[i][1] != 0:
                sobra_c += demanda_mercado[i][1]
        
            if demanda_mercado[i][1] > 0:
                veces_mercado_c_no_cumple += 1
                suma_resto_mercado_c_positiva_solamente += demanda_mercado[i][1]
        elif demanda_mercado[i][0] == "demanda_mercado_d":
            if demanda_mercado[i][1] != 0:
                sobra_d += demanda_mercado[i][1]
        
            if demanda_mercado[i][1] > 0:
                veces_mercado_d_no_cumple += 1
                suma_resto_mercado_d_positiva_solamente += demanda_mercado[i][1]

Calidad máxima: 1
Calidad mínima: 0.824690825

Litros totales obtenidos: 45821250
Botellas totales obtenidas: 61095000
Costos totales esperadas USD (considerando contratos spot y foward): 40216577

Litros totales perdidos: 2358750
Botellas totales perdidas: 3145000
Costos totales perdidos USD (compra spot): 1846133

Costos totales etapa 1: 42062710
Costos recuperados etapa 2: 93858 USD
Costos totales etapa 2, post recuperación: 41968852
Se perdieron 9 tanques el día 77
9
Con el costo de 259199.99999999997 USD

Se perdieron 8 tanques el día 77
8
Con el costo de 221600.00000000003 USD

Se perdieron 11 tanques el día 77
11
Con el costo de 195250.0 USD

Se perdieron 9 tanques el día 77
9
Con el costo de 238050.0 USD

Se perdieron 3 tanques el día 77
3
Con el costo de 82050.0 USD

Costos perdidos por no entrar a tanques: 1325769
Costos totales hasta etapa 3 fermentación: 41985022

RESULTADOS HASTA FERMENTACIÓN
litros perdidos por cepas (c1 a c6): [1073125.0, 335000.0, 644375.0, 300000.0, 25

In [13]:
def calcular_promedio(lista):
    return sum(lista) / len(lista)
print(g)
# Sobra promedio considerando solo las veces en que hay sobra
print(sobra_a/1000,"A")
print(sobra_b/1000, "b")
print(sobra_c/1000, "C")
print(sobra_d/1000, "D")
print(veces_mercado_a_no_cumple)
print(veces_mercado_b_no_cumple)
print(veces_mercado_c_no_cumple)
print(veces_mercado_d_no_cumple)
print("sobra")
#sobra o exceso promedio sumando las veces en que sobra y las veces en que excedo la demanda minima sacando un promedio general por mercado
print(suma_resto_mercado_a_positiva_solamente/1000)
print(suma_resto_mercado_b_positiva_solamente/1000)
print(suma_resto_mercado_c_positiva_solamente/1000)
print(suma_resto_mercado_d_positiva_solamente/1000)
print(min(calidad_minima_por_simulacion), "MINIMA CALIDAD DE TODA LA SIMULACION")
#aqui no se como sacar los litros totales perdidos de todas estas variables habria que ver si se suman estos tres nomas o quizas el tipo3 etapa2 ya considera los de la etapa 1 nose 
#hay que ver pero corra la sim y ahi cachamos cuales sumamos nomas
print(litros_totales_perdidos_etapa1/1000,"litros_totales_perdidos_etapa1")
PC1 = 0
PC2 = 0
PC3 = 0
PC4 = 0
PC5 = 0
PC6 = 0

for i in litros_totales_perdidos_etapa2:
    PC1 += i[0]
    PC2 += i[1]
    PC3 += i[2]
    PC4 += i[3]
    PC5 += i[4]
    PC6 += i[5]
print(PC1/1000,"Pc1")
print(PC2/1000,"Pc2")
print(PC3/1000,"Pc3")
print(PC4/1000,"Pc4")
print(PC5/1000,"Pc5")
print(PC6/1000,"Pc6")

PC1_3 = 0
PC2_3 = 0
PC3_3 = 0
PC4_3 = 0
PC5_3 = 0
PC6_3 = 0

for i in litros_totales_perdidos_etapa2_tipo3:
    PC1_3 += i[0]
    PC2_3 += i[1]
    PC3_3 += i[2]
    PC4_3 += i[3]
    PC5_3 += i[4]
    PC6_3 += i[5]

print(PC1_3/1000,"Pc1_3")
print(PC2_3/1000,"Pc2_3")
print(PC3_3/1000,"Pc3_3")
print(PC4_3/1000,"Pc4_3")
print(PC5_3/1000,"Pc5_3")
print(PC6_3/1000,"Pc6_3")


1
-328.835 A
-1175.418 b
365.831 C
0.0 D
0
0
1
0
sobra
0.0
0.0
365.831
0.0
0.824690825 MINIMA CALIDAD DE TODA LA SIMULACION
2358.75 litros_totales_perdidos_etapa1
1073.125 Pc1
335.0 Pc2
644.375 Pc3
300.0 Pc4
259.375 Pc5
1062.5 Pc6
525.0 Pc1
305.625 Pc2
361.875 Pc3
247.5 Pc4
185.625 Pc5
733.125 Pc6


In [14]:
print(ingresos_totales_finales)
print(costos_totales_finales)
print(utilidad_total_final)
print(calcular_promedio(ingresos_totales_finales))
print(calcular_promedio(costos_totales_finales))
print(calcular_promedio(utilidad_total_final))
#lista de las demandas por mercado por cada sim
print(demandas_por_mercado)
print("Cantidad promedio de uva de cada cepa desde C1 a C6 que sale de la etapa de fermentacion hacia mezcla blends")
print(C1_fermentacion_suma/1000)
print(C2_fermentacion_suma/1000)
print(C3_fermentacion_suma/1000)
print(C4_fermentacion_suma/1000)
print(C5_fermentacion_suma/1000)
print(C6_fermentacion_suma/1000)
#Calcula el promedio que se ocupo de cada cepa 
C1 = 0
C2 = 0
C3 = 0
C4 = 0
C5 = 0
C6 = 0
Blend1 = 0
Blend2 = 0
Blend3 = 0
Blend4 = 0

for i in cepa_inicio_total:
    C1 += i[0]
    C2 += i[1]
    C3 += i[2]
    C4 += i[3]
    C5 += i[4]
    C6 += i[5]
    Blend1 += i[6]
    Blend2 += i[7]
    Blend3 += i[8]
    Blend4 += i[9]
print(C1/1000)
print(C2/1000)
print(C3/1000)
print(C4/1000)
print(C5/1000)
print(C6/1000)
print(Blend1/1000)
print(Blend2/1000)
print(Blend3/1000)
print(Blend4/1000)


[153330880]
[147350616]
[5980264]
153330880.0
147350616.0
5980264.0
[[9848835.0, 13852254.0, 10740836.0, 9507169.0]]
Cantidad promedio de uva de cada cepa desde C1 a C6 que sale de la etapa de fermentacion hacia mezcla blends
8256.25
7453.75
5641.25
5742.5
10351.25
6491.25
7060.336
6799.169
4959.418
5116.166
9848.835
4074.337
316.666
750.0
1082.5
3941.667
